<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebooks/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Progetto Baseline

## 1. Introduzione al Progetto

### 1.1. Obiettivo del Modello

L'obiettivo di questo progetto è sviluppare un modello di regressione in grado di stimare il punteggio ELO del giocatore con i pezzi bianchi a partire da feature estratte da una singola partita.
Questo modello, denominato modello baseline, sarà successivamente confrontato con un secondo modello ... [DA COMPLETARE]

### 1.2. Limiti del Modello

**A. Rumore nei dati**

Questo modello stima il punteggio ELO di un giocatore utilizzando esclusivamente dati provenienti da una singola partita. Poiché la performance in una singola partita può variare significativamente rispetto al livello medio del giocatore (a causa di fattori quali concentrazione, forma del giorno, errori occasionali, o scelte strategiche particolari), stiamo di fatto lavorando con un campione altamente rumoroso dell'abilità reale del giocatore. Inoltre, il modello baseline utilizza feature non direttamente indicative della qualità del gioco (come vedremo nella sezione delle feature).

Per questi motivi, non ci aspettiamo un modello estremamente preciso e valuteremo criticamente eventuali metriche di accuratezza eccessivamente elevate, che potrebbero indicare overfitting o altre problematiche.

**B. Importanza delle feature e information leakage**

Tra le feature utilizzate (presentate nel punto 1.4 e successivamente selezionate nella sezione 2.1/b), ce n'è una chiaramente più informativa delle altre: `black_rating`. Questo deriva dal meccanismo di matchmaking utilizzato dai siti di scacchi online, che tendono ad accoppiare giocatori con punteggi ELO simili per garantire partite equilibrate.
(Questo non è vero universalmente, all'interno di tornei o partite amichevoli questa regola può non valere).

Di conseguenza, il modello avrà inevitabilmente una struttura approssimabile a:

**white_rating ≈ black_rating ± f(altre feature)**

dove f rappresenta una funzione correttiva basata sulle altre feature disponibili. Questa situazione rappresenta un caso di information leakage parzialmente controllato: `black_rating` contiene informazioni altamente correlate con il target, ma non lo determina completamente, lasciando spazio alle altre feature per catturare le differenze individuali e contestuali.

**C. Partite classificate vs non classificate**

È ragionevole ipotizzare che la performance del modello possa essere inferiore per le partite non classificate (rated=False), ovvero quelle che non influenzano il punteggio ELO dei giocatori. Il motivo è che molti giocatori potrebbero affrontare queste partite con minore serietà o utilizzarle per sperimentare strategie non convenzionali, dato che un'eventuale sconfitta non avrebbe conseguenze sul loro rating ufficiale.

Questa ipotesi potrà essere verificata attraverso un'analisi comparativa delle performance del modello sui due sottoinsiemi di dati.

### 1.3. Note sul Target

Il target del modello, `white_rating`, rappresenta il punteggio ELO del giocatore che conduce i pezzi bianchi. Il sistema ELO (https://it.wikipedia.org/wiki/Elo_(scacchi)) è un metodo per calcolare il livello relativo di abilità dei giocatori in giochi a due giocatori a somma zero, come gli scacchi.

**Aspetto temporale cruciale**: È fondamentale specificare che `white_rating` si riferisce al punteggio che il giocatore possedeva immediatamente prima dell'inizio della partita, non dopo la sua conclusione. Nelle partite classificate (rated=True), il punteggio ELO viene aggiornato al termine della partita in base a:
- Il rating pre-partita di entrambi i giocatori
- Il risultato effettivo della partita
- Il risultato atteso (calcolato dalla differenza di rating)
- Altri parametri come il fattore K (che dipende dal numero di partite giocate)

Il nostro obiettivo non è invertire l'algoritmo di calcolo del punteggio ELO, bensì utilizzare tale punteggio come proxy (approssimatore imperfetto ma ragionevole) del livello di abilità scacchistica del giocatore in quel momento specifico.

### 1.4. Presentazione del Dataset

Il dataset utilizzato proviene da Kaggle ed è disponibile al seguente indirizzo:
https://www.kaggle.com/datasets/datasnaek/chess

Il dataset originale contiene 20.058 partite di scacchi disputate sulla piattaforma Lichess. Ogni partita è descritta dalle seguenti variabili:

- **id**: Stringa alfanumerica identificativa univoca della partita
- **rated**: Variabile booleana che indica se la partita è classificata (True) e quindi influenza il rating ELO dei giocatori, oppure non classificata (False)
- **created_at**: Timestamp di inizio della partita in formato Unix time
- **last_move_at**: Timestamp di conclusione della partita in formato Unix time
- **turns**: Numero di turni completi, dove ogni turno è definito come una coppia di mosse (bianco-nero)
- **victory_status**: Modalità di conclusione della partita (scacco matto, abbandono, tempo esaurito, stallo, ecc.)
- **winner**: Colore del giocatore vincente (white, black) o draw in caso di patta
- **increment_code**: Formato temporale della partita espresso come "minuti iniziali+secondi di incremento per mossa" (es. "10+0" indica 10 minuti per giocatore senza incremento)
- **white_id**: Nome utente del giocatore con i pezzi bianchi
- **white_rating**: Punteggio ELO del giocatore bianco (pre-partita)
- **black_id**: Nome utente del giocatore con i pezzi neri
- **black_rating**: Punteggio ELO del giocatore nero (pre-partita)
- **moves**: Sequenza completa delle mosse della partita in notazione algebrica standard (SAN - Standard Algebraic Notation)
- **opening_eco**: Codice di classificazione dell'apertura secondo l'Encyclopaedia of Chess Openings (ECO)
- **opening_name**: Nome descrittivo dell'apertura utilizzata
- **opening_ply**: Numero di ply (mezze-mosse) che costituiscono la fase di apertura classificata della partita


### 1.5. Importazione del Dataset

In [2]:
###INSERIRE CODICE PER IMPORTAZIONE

In [3]:
## Quando ultimeremo il codice sarebbe meglio non fare montare il drive.

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df=pd.read_csv("/content/drive/MyDrive/games.csv")
df_originale = df.copy() # Salviamo una copia del DataFrame originale

Mounted at /content/drive


## 2. Sviluppo del Modello

### 2.1. Pulizia e Pre-processing dei Dati

#### a) Rimozione dei duplicati

Durante l'ispezione iniziale del dataset è stata rilevata la presenza di 945 record con identificativo (`id`) duplicato. Dopo aver verificato che si trattasse effettivamente di righe identiche in tutte le colonne, e non di partite diverse erroneamente associate allo stesso ID, abbiamo proceduto alla rimozione delle righe ridondanti, mantenendo una sola istanza per ciascun ID univoco.

In [4]:
# Individuiamo gli ID che compaiono più di una volta
dup_ids = df['id'].value_counts()
dup_ids = dup_ids[dup_ids > 1]

print(f"Numero di ID duplicati: {len(dup_ids)}")

colonne_da_controllare = [c for c in df.columns if c != 'id']

identical_check = (
    df[df['id'].isin(dup_ids.index)]
    .groupby('id')[colonne_da_controllare]
    .nunique()
    .max(axis=1)
)

identical_check.value_counts()

Numero di ID duplicati: 813


,count
2,516
1,297


In [5]:
# Notiamo che alcune righe (516) non condividono valori identici per tutte le colonne, verifichiamo perché:
id_problematici = identical_check[identical_check > 1].index

df[df['id'].isin(id_problematici)] \
  .groupby('id')[colonne_da_controllare] \
  .nunique() \
  .head()

# Le colonne non identiche sono quelle di inizio e fine partita, dunque il problema
# è probabilmente dovuto a un problema di datatype o di misurazione nel dataset.
# Le partite sono semanticamente identiche.
# Possiamo quindi procedere con la rimozione dei duplicati.

df = df.drop_duplicates(subset='id').reset_index(drop=True)

b) *Eliminazione delle colonne del dataframe non necessarie*:
Saranno certamente ininfluenti gli ID dei giocatori, ID della partita e il nome dell'apertura (prediligeremo usare il più rigoroso opening ECO).
Il discorso è più complicato per quanto riguarda le mosse. Intuitivamente per valutare il livello di un giocatore queste sarebbero fondamentali, tuttavia, almeno per il modello baseline, proveremo a fare le nostre valutazioni senza prenderle in considerazione.

In [6]:
df = df.drop(columns=['white_id','black_id','id','opening_name','moves'])

c) *Trasformazione delle colonne di inizio e fine partita in una colonna di durata*

In [7]:
#Ora convertiamo le colonne di inizio e fine partita, che sono in formato Unix time, in formato datetime
#Dopodichè inseriremo una più semplice colonna di durata partita al posto delle due colonne iniziali

df_times = df[['created_at','last_move_at']].copy()

df_times['created_at_dt'] = pd.to_datetime(df_times['created_at']/1000, unit='s', origin='unix')

df_times['last_move_at_dt'] = pd.to_datetime(df_times['last_move_at']/1000, unit='s', origin='unix')

df_times['created_time'] = df_times['created_at_dt'].dt.time
df_times['last_move_time'] = df_times['last_move_at_dt'].dt.time
df_times['duration_seconds'] = (df_times['last_move_at_dt'] - df_times['created_at_dt']).dt.total_seconds()

#Adesso che abbiamo la durata della partita in secondi, inseriamo la colonna nel dataframe e rimuoviamo quelle iniziali

df=df.drop(columns=['created_at','last_move_at'])
df=df.join(df_times['duration_seconds'])

#Ora invece ci occupiamo della colonna 'increment_code', che è della forma "X+Y"
#dove X sono i minuti concessi a ciascun giocatore per giocare le proprie mosse e Y
#è il tempo che gli viene riaggiunto una volta giocata la mossa.
#Vogliamo separare questi due valori e tenerli in colonne separate

#Split di increment_code in due colonne
df[['base_time', 'increment_time']] = (
    df['increment_code'].str.split('+', expand=True).astype(int))

#Rimuoviamo la colonna originale
df = df.drop(columns=['increment_code'])

df.tail(50)

,rated,turns,victory_status,winner,white_rating,black_rating,opening_eco,opening_ply,duration_seconds,base_time,increment_time
19063,True,119,draw,draw,2049,1895,D31,5,1895.975,15,5
19064,True,118,resign,black,1798,2044,B07,2,2231.065,15,5
19065,True,58,resign,black,2001,2030,B07,4,1819.828,15,5
19066,True,60,resign,black,1980,2017,A03,2,629.199,5,5
19067,True,63,outoftime,white,2010,1781,A49,5,813.539,5,5
19068,True,87,outoftime,white,1998,1920,E20,8,782.868,5,5
19069,True,50,resign,black,1684,1993,D02,5,488.695,7,2
19070,True,29,resign,white,1989,1653,D19,15,315.656,7,2
19071,True,46,mate,black,1838,1979,B09,10,409.982,7,2
19072,True,41,mate,white,1972,1718,D31,6,277.190,7,2


d) Gestione delle variabili categoriche:

- Iniziamo occupandoci dell'opening ECO. Questo è un sistema complesso. Per i nostri scopi sarà sufficiente sapere che classifica le aperture in cinque grandi famiglie:
A -> Aperture atipiche; B -> Sistemi semiaperti; C -> Partite di Re aperte e Francesi;  D -> Partite di Donna;                             E -> Difese Indiane.    
Questa è un'approsimazione ma ci consente di creare comunque dei dummy ristretti ma informativi riguardo al tipo di apertura giocata.

In [11]:
# Opening ECO semplificato

df['opening_category'] = df['opening_eco'].str[0]
df = df.drop(columns=['opening_eco'])

colonne_categoriali = ['rated', 'victory_status', 'winner', 'opening_category']

df = pd.get_dummies(df, columns=colonne_categoriali, dtype=int, drop_first=False)

KeyError: 'opening_eco'